# Cleaning the data for KNN+Regression for Question 3

We will be performing this twice for the months

The first time will both types of customers, tippers and non-tippers. The second time will include only those who paid tips.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
df=pd.read_csv("datasets/yellow_tripdata_2013-01.csv")

In [4]:
df = df.loc[(df['pickup_longitude'] > -74.06) & (df['pickup_longitude'] < -73.77) & (df['pickup_latitude'] > 40.61) &  (df['pickup_latitude'] < 40.91)]
df = df.loc[(df['tip_amount'] > 0.0)] #comment this line out to include those who don't pay tips
df = df.reset_index()
df = df.drop('index', 1)
df = df.drop('vendor_id',1)
df = df.drop('rate_code',1)
df = df.drop('store_and_fwd_flag',1)
df = df.drop('fare_amount',1)
df = df.drop('surcharge',1)
df = df.drop('mta_tax',1)
df = df.drop('tolls_amount',1)
df = df.drop('dropoff_latitude',1)
df = df.drop('dropoff_longitude',1)
df = df.drop('passenger_count',1)
df = df.drop('payment_type',1)

In [5]:
def add_data(df):
    df_timestamp = pd.to_datetime(pd.Series(df['pickup_datetime']))
    df['trip_distance']*0.621371 # convert to miles
    df['weekday'] = df_timestamp.dt.weekday_name
    #df['month'] = df_timestamp.dt.month
    df['hour'] = df_timestamp.dt.hour
    #df['day'] = df_timestamp.dt.day
    #df['minutes'] = (df_timestamp.dt.hour)*60 + df_timestamp.dt.minute
    time_spent = pd.to_datetime(df['dropoff_datetime']) - pd.to_datetime(df['pickup_datetime'])
    df['time_spent'] = pd.to_datetime(time_spent).dt.minute
    df['pickup'] = df['pickup_latitude'].map(str) +','+df['pickup_longitude'].map(str)
    return df

In [6]:
df = add_data(df)

In [7]:
df = df.drop('pickup_datetime',1)
df = df.drop('dropoff_datetime',1)
df = df.drop('pickup_longitude',1)
df = df.drop('pickup_latitude',1)

In [8]:
df.head()

,trip_distance,tip_amount,total_amount,weekday,hour,time_spent,pickup
0,2.92,2.75,14.25,Sunday,10,8,"40.723945,-73.99218999999998"
1,3.93,2.40,14.90,Sunday,10,9,"40.731427,-73.98238"
2,4.51,2.90,17.90,Sunday,10,10,"40.77718,-73.957385"
3,1.84,1.70,10.70,Sunday,10,7,"40.705572,-74.00825"
4,20.78,7.70,65.00,Sunday,10,36,"40.641513,-73.78830999999998"


In [9]:
# Look into the dataframe 
# First by weekday, then hour then block
# and we will know the average tip amount
# for each weekday > hour > block
# eg. On (day of week) at (hour) on (lat,long) avg tip is $number
def get_avg_tips(df):
    avg_tips = df.groupby(['weekday','hour','pickup']).mean()
    avg_tips = avg_tips.reset_index()
    return avg_tips

In [10]:
df = get_avg_tips(df)

In [11]:
df.shape

(7375729, 7)

In [12]:
df.head()

,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
0,Friday,0,"40.613763,-73.972592",0.00,7.70,38.50,0.0
1,Friday,0,"40.61622,-73.97454999999998",2.90,3.25,16.25,11.0
2,Friday,0,"40.620231,-73.96423799999998",5.10,5.00,25.00,19.0
3,Friday,0,"40.629405,-74.017868",13.36,7.50,45.50,22.0
4,Friday,0,"40.63121,-74.017517",2.64,2.20,13.70,11.0


In [13]:
df.tail()

,weekday,hour,pickup,trip_distance,tip_amount,total_amount,time_spent
7375724,Wednesday,23,"40.873275,-73.886922",2.8,1.50,13.50,10.0
7375725,Wednesday,23,"40.890857,-73.908495",0.0,4.41,7.91,1.0
7375726,Wednesday,23,"40.900912,-74.00320499999998",0.0,23.00,75.50,0.0
7375727,Wednesday,23,"40.902505,-74.00228799999998",0.0,3.00,86.00,6.0
7375728,Wednesday,23,"40.907027,-73.909115",0.0,9.50,50.50,0.0


We will call the cleaned csv with those who didn't pay tips clean-january-2013.csv

We will call the cleaned csv with only those who did pay tips cleaner-january-2013.csv

In [14]:
#df.to_csv('clean-january-2013.csv',sep=',',encoding='utf-8') # both types
df.to_csv('cleaner-january-2013.csv',sep=',',encoding='utf-8') # for only those who gave tips